<a href="https://colab.research.google.com/github/Harbhajan-Kaur/final_project_Harbhajan_Kaur/blob/main/Kaggle_import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
! mkdir ~/.kaggle

In [7]:
!cp /content/drive/MyDrive/Kaggle_API/kaggle.json ~/.kaggle/

In [8]:
! chmod 600 ~/.kaggle/kaggle.json

In [9]:
! kaggle datasets download ankushpanday1/tuberculosis-tb-predictiontop-75-countries

Dataset URL: https://www.kaggle.com/datasets/ankushpanday1/tuberculosis-tb-predictiontop-75-countries
License(s): MIT
 88% 55.0M/62.2M [00:00<00:00, 145MB/s]
100% 62.2M/62.2M [00:00<00:00, 135MB/s]


In [11]:
! unzip tuberculosis-tb-predictiontop-75-countries.zip

Archive:  tuberculosis-tb-predictiontop-75-countries.zip
  inflating: Tuberculosis_Dataset.csv  


In [12]:
import pandas as pd

In [14]:
data= pd.read_csv('/content/Tuberculosis_Dataset.csv')

In [15]:
data

,Country,Year,TB_Incidence_Rate,Population,HIV_Prevalence,Treatment_Success_Rate,Mortality_Rate,Urban_Population_Percentage,Smoking_Rate,Alcohol_Consumption,...,Age_Group,Gender,HIV_CoInfection_Rate,BCG_Vaccination_Coverage,Multidrug_Resistance_Rate,Unemployment_Rate,GDP_Per_Capita,Life_Expectancy,TB_Screening_Coverage,Rural_Population_Percentage
0,Country_52,2003,56.145286,788566,15.124856,63.008939,18.417301,39.939777,38.323440,13.115522,...,25-54,Female,4.491676,87.163577,1.405499,27.964788,98235.258489,41.642988,53.486072,89.256182
1,Country_15,2021,137.470346,1282706,1.365238,63.861596,25.804928,78.360506,48.317584,13.676533,...,0-14,Female,0.527789,61.848695,4.022342,29.374837,35130.172901,63.968269,76.473413,42.549955
2,Country_61,2014,179.814220,1100374,13.770958,92.017009,15.319141,27.609710,9.085422,3.070677,...,65+,Female,0.977231,63.133755,14.618375,23.704493,24703.776975,81.883098,59.468310,29.938714
3,Country_21,2014,124.211205,1141641,8.198825,94.747900,9.405584,73.239525,39.927303,7.360397,...,65+,Male,3.218930,55.069057,5.758827,21.854778,16417.483760,64.295034,96.677854,32.578901
4,Country_24,2011,50.394259,759040,4.472235,83.710431,37.830973,11.821832,48.280581,4.958237,...,55-64,Female,1.938210,70.991237,0.201211,23.512661,60373.614322,52.874425,88.048894,17.109448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,Country_53,2004,169.787616,660860,10.276245,78.481571,22.047078,22.734980,33.680575,9.620027,...,25-54,Male,1.584840,94.925027,6.234854,10.147965,90021.693437,57.804647,75.950046,41.995757
399996,Country_46,2009,193.569717,1189183,3.147915,68.693823,29.024622,13.694117,31.240885,5.127701,...,55-64,Male,2.972354,80.601166,4.926648,1.937516,58162.817946,68.721109,88.876142,43.274853
399997,Country_8,2019,180.934211,1031969,14.985351,88.738280,14.540239,45.215312,44.383766,9.909989,...,25-54,Male,1.775331,52.275679,6.439025,4.549042,6778.323905,66.494715,79.835039,64.421630
399998,Country_49,2021,207.300043,1076404,5.876641,69.758055,18.467550,81.458567,27.242929,6.147466,...,15-24,Male,2.377095,54.302045,16.140677,29.894668,61021.649800,82.612641,57.225886,12.379654
